# Bayesian Data Analysis - Project

In [1]:
%%capture
import pystan
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, Math, Latex

### Separate model

In [2]:
code_separate = """
data {
    int<lower=0> N;  // number of data points
    int<lower=0> K;  // number of countries
    //int<lower=0, upper=K> ind[N]; // country indicator
    vector[N] x;     // observation year
    matrix[K, N] y;  // observation life expectancy
    real xpred;      // prediction year
}
parameters {
    real alpha[K];
    real beta[K];
    real<lower=0> sigma[K];
}
transformed parameters {
    vector[N] mu[K];
    for (k in 1:K)
        mu[k] = alpha[k] + beta[k]*x;
}
model {
    //beta ~ normal(0, 18.5483); //assign priors here
    for (k in 1:K)
        y[k] ~ normal(mu[k], sigma[k]);
}
generated quantities {
    real ypred[K];
    for (k in 1:K)
        ypred[k] = normal_rng(alpha[k]+beta[k]*xpred, sigma[k]);
}
"""
sm_separate = pystan.StanModel(model_code=code_separate)

/opt/conda/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp53rji0qo/stanfit4anon_model_b8f7694347c95a61b77f77136d474d39_75790019742490364.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


We can then sample from the model with the data.

In [3]:
import numpy as np
year_list = []
life_expectancy_data = []
with open("lifex.csv", "r") as ifile:
    year_list = [int(x) for x in ifile.readline().split(",")[1:-1]]
    for line in ifile:
        life_expectancy_data.append([float(x) for x in line.split(",")[1:-1]])

life_expectancy_data = np.array(life_expectancy_data)
N = life_expectancy_data.shape[1]
K = life_expectancy_data.shape[0]
ind = [x+1 for x in range(life_expectancy_data.shape[0]) for _ in range(life_expectancy_data.shape[1])]
y = life_expectancy_data
x = year_list
stan_data = dict(N=N, K=K, x=x, y=y, ind=ind, xpred=2017)


In [5]:
fit = sm_separate.sampling(data=stan_data, iter=4000); #, iter=4000
print(fit)

Inference for Stan model: anon_model_b8f7694347c95a61b77f77136d474d39.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[1] -400.7    0.36   5.71 -412.3 -404.2 -400.6 -397.1 -389.3    248    1.0
alpha[2] -53.34    1.26  11.24 -75.44 -60.77 -53.24 -46.23 -30.85     79   1.04
alpha[3] -389.1    5.47  32.21 -424.5 -404.1 -392.8 -381.7 -301.2     35   1.07
alpha[4] -409.4    0.08   7.06 -423.3 -414.1 -409.4 -404.6 -395.6   8128    1.0
alpha[5] -372.2    2.67  17.99 -386.1 -377.8 -374.2 -370.4 -359.1     45   1.07
beta[1]    0.24  1.8e-4 2.9e-3   0.23   0.24   0.24   0.24   0.25    248    1.0
beta[2]    0.06  6.3e-4 5.7e-3   0.05   0.06   0.06   0.07   0.07     79   1.04
beta[3]    0.23  2.8e-3   0.02   0.19   0.23   0.23   0.24   0.25     35   1.07
beta[4]    0.24  3.9e-5 3.6e-3   0.24   0.24   0.24   0.25   0.25   8103    1.0
beta[5]    

### Pooled model
absolutely useless in this case

In [6]:
code_pooled = """
data {
    int<lower=0> N;  // number of data points
    int<lower=0> K;  // number of countries
    //int<lower=0, upper=K> ind[N]; // country indicator
    vector[N] x;     // observation year
    matrix[K, N] y;  // observation life expectancy
    real xpred;      // prediction year
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
transformed parameters {
    vector[N] mu;
    mu = alpha + beta*x;
}
model {
    //beta ~ normal(0, 18.5483); //assign priors here
    for (k in 1:K)
        y[k] ~ normal(mu, sigma);
}
generated quantities {
    real ypred[K];
    for (k in 1:K)
        ypred[k] = normal_rng(alpha+beta*xpred, sigma);
}
"""
sm_pooled = pystan.StanModel(model_code=code_pooled)

/opt/conda/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpsu9v5fj4/stanfit4anon_model_a032551cfd11e42adfb10f4c00f3c6d4_1855022194978445426.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [7]:
fit = sm_pooled.sampling(data=stan_data);
print(fit)

Inference for Stan model: anon_model_a032551cfd11e42adfb10f4c00f3c6d4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha    -325.7    0.46  15.19 -355.2 -335.9 -326.0 -315.4 -295.0   1098    1.0
beta        0.2  2.3e-4 7.6e-3   0.19    0.2    0.2   0.21   0.22   1098    1.0
sigma      2.09  2.2e-3   0.09   1.92   2.03   2.08   2.14   2.27   1548    1.0
mu[1]     68.61  6.7e-3   0.24  68.12  68.44   68.6  68.76  69.09   1341    1.0
mu[2]     68.81  6.5e-3   0.24  68.33  68.65  68.81  68.96  69.27   1358    1.0
mu[3]     69.01  6.2e-3   0.23  68.55  68.85  69.01  69.16  69.46   1377    1.0
mu[4]     69.21  6.0e-3   0.23  68.76  69.06  69.21  69.35  69.65   1398    1.0
mu[5]     69.41  5.8e-3   0.22  68.97  69.27  69.41  69.55  69.84   1421    1.0
mu[6]     69.61  5.6e-3   0.21  69.19  69.47  69.61  69.75  70.03   1447    1.0
mu[7]     6

### Hierarchical model

In [ ]:
code_hierarchical = """
data {
    int<lower=0> N;  // number of data points
    int<lower=0> K;  // number of countries
    //int<lower=0, upper=K> ind[N]; // country indicator
    vector[N] x;     // observation year
    matrix[K, N] y;  // observation life expectancy
    real xpred;      // prediction year
}
parameters {
    real alpha[K];
    real beta[K];
    real<lower=0> sigma[K];
}
transformed parameters {
    vector[N] mu[K];
    for (k in 1:K)
        mu[k] = alpha[k] + beta[k]*x;
}
model {
    //beta ~ normal(0, 18.5483); //assign priors here
    for (k in 1:K)
        y[k] ~ normal(mu[k], sigma[k]);
}
generated quantities {
    real ypred[K];
    for (k in 1:K)
        ypred[k] = normal_rng(alpha[k]+beta[k]*xpred, sigma[k]);
}
"""
sm_hierarchical = pystan.StanModel(model_code=code_hierarchical)